# Using GPT4 to answer D&D questions


D&D has a lot of rules. This notebook uses GPT4 and a vector database to answer D&D questions.

To setup locally run:
```bash
python -m venv venv
source ./venv/bin/activate
pip install -r requirements.txt
```
The source line is shell and platform dependant. For example fish is `source ./venv/bin/activate.fish`

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from sentence_transformers import SentenceTransformer
from typing import List, Callable
import uuid
import os
import re
from functools import partial
import openai

# Load Data
First we need the rules as text. In this example we load the SRD from a helpful GitHub repository that has converted it to Markdown. You could replace this with whatever rules you can get your hands on.

In [2]:
# Checkout github.com/OldManUmby/DND.SRD.Wiki shallow clone
# Tested with 7c99bcc60bb67c066b1b59a3d20f52278371fd48
datadir = "./srd"
if not os.path.isdir(datadir):
    os.system(f"git clone --depth 1 https://github.com/OldManUmby/DND.SRD.Wiki {datadir}")

# Cleanup the stuff we dont' need like the (Alt folders and the readme)
os.system(f"rm -rf {datadir}/*Alt*")
os.system(f"rm -rf {datadir}/.git")
os.system(f"rm -rf {datadir}/.github")
os.system(f"rm -rf {datadir}/*.md")
os.system(f"rm -rf {datadir}/*.png")

0

# Reading markdown files
Next we just need to load the markdown files.

In [3]:
def readMarkdownDirectoryRecursive(directory: str) -> List[str]:
    markdown_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".md"):
                markdown_files.append(os.path.join(root, file))
    return list(map(readWholeFile, markdown_files))

def readWholeFile(filename: str):
        with open(filename, 'r') as file:
                return file.read()

srdMdFiles = readMarkdownDirectoryRecursive(datadir)

# Chunking text
Splitting the Markdown into chucks is a bit of a fiddly step. The size and quality of chunking are important for the embeddings model to be able to understand what is going on.

Here we split on Markdown headings, then newlines, then we have to give up and split wherever. This seems to work well enough for the SRD text we are using here. 

In [4]:
# Splits a string closets to the middle as possble keeping the delimiter
def split_middle(splitstr: str, item: str, maxlen: int) -> List[str]:
    if len(item) < 8:
        return [item]

    half = len(item) // 2
    left = item[:half]
    right = item[half:]
    try:
        loc_left = left.rindex(splitstr)
    except ValueError:
        loc_left = None
    try:
        loc_right = right.index(splitstr)
    except ValueError:
        loc_right = None
        
    # No split found
    if loc_left == None and loc_right == None:
        return [item]
        
    # if left is closer than right or right is disqualified
    if loc_right == None or (loc_left != None and (len(left) - loc_left) < loc_right):
        split_left = left[:loc_left]
        split_right = left[loc_left:] + right
    else: # Otherwise right is closer than left
        split_left = left + right[:loc_right]
        split_right = right[loc_right:]
    
    # Prevent tiny splits.
    if len(split_left) < 2 or len(split_right) < 2:
        return [item]
        
    return [split_left, split_right]

def split_recursive(items: List[str], maxlen: int, splitLogic: Callable[[str, int], List[str]]) -> List[str]:
    result = []
    for item in items:
        if len(item) > maxlen:
            split = splitLogic(item, maxlen)
            # Base case, no possible separator.
            if len(split) == 1:
                result += [item]
            else:
                result += split_recursive(split, maxlen, splitLogic)
        else:
            result += [item]
    return result
    
def make_splitter(split_pattern: str) -> Callable[[str, int], List[str]]:
    return partial(split_middle, split_pattern)
    
def split_markdown_on_headings(items: List[str], maxlen: int) -> List[str]:
    result = items
    for depth in range(1, 7):
        separator = "\n" + "#"*depth + " "
        result = split_recursive(result, maxlen, make_splitter(separator))
    return result

def split_markdown(text: List[str], maxlen: int) -> List[str]:
    table_start = re.compile(r"(\|.*\|)")
    
    # Start with the whole text
    splits = text
    
    # Split on headings
    splits = split_markdown_on_headings(splits, maxlen)

    # Split on Newlines
    splits = split_recursive(splits, maxlen, make_splitter("\n"))
    
    # Trim whitespace
    splits = list(map(str.strip, splits))
    
    # Last resort trim on whatever. This guarenttes no enteries over a certain value
    splits = split_recursive(splits, maxlen, make_splitter(""))
    splits = list(map(str.strip, splits))

    return splits


In [5]:
splitMd = split_markdown(srdMdFiles, 1024)

# Create Embeddings
Next we create some embeddings. The embeddings allow us to do a symantic search for the users question within the rules. We will then store them in a vector database for easy retrieval in the next step. 

For this example we use sentance transformers embeddings. Picked because it was a good general purpose model. We could also use openai's embeddings model here if we don't want to wait around for them to compute locally.

If you want to run this example and don't have a GPU. Remove the `device='cuda'` parameter. 

In [6]:
class Embeddor:
    def embeddings(self, text: List[str]) -> List[List[float]]:
        pass

    def countTokens(self, text: List[str]) -> List[int]:
        pass

    def getEmbeddingSize(self) -> int:
        pass
        
    def getModelName(self) -> str:
        pass

class SentenceTransformerEmbeddor(Embeddor):
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.model = SentenceTransformer(model_name, device='cuda')
        self.embedding_size = self.model.get_sentence_embedding_dimension()

    def embeddings(self, text: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(text)
        return [embedding.tolist() for embedding in embeddings]

    def countTokens(self, text: str) -> List[int]:
        return len(self.model.tokenize([text])['input_ids'][0])
    
    def max_sequence_length(self) -> int:
        return self.model.max_seq_length
    
    def count_oversized(self, text: List[str]) -> int:
        count = 0
        max_length = self.max_sequence_length()
        for item in splitMd:
            num_tokens = self.countTokens(item)
            if num_tokens >= max_length:
                count += 1

    def getEmbeddingSize(self) -> int:
        return self.embedding_size
        
    def getModelName(self) -> str:
        return self.model_name


In [7]:
embeddor = SentenceTransformerEmbeddor("sentence-transformers/multi-qa-mpnet-base-dot-v1")
embeddings = embeddor.embeddings(splitMd)

# Store embeddings in vector database

Next we store the embeddings we create in a vector database. The specifics of how to use the vector database are not important here and there are many options for vector databases. We use Qdrant here because it's convenient. 

In [8]:
client = QdrantClient(":memory:")
srd_collection_name = "srd"
client.create_collection(collection_name=srd_collection_name, vectors_config=VectorParams(size=embeddor.getEmbeddingSize(), distance=Distance.COSINE))
points = [PointStruct(id=str(uuid.uuid4()), vector=embedding, payload={"text": text}) for embedding, text in zip(embeddings, splitMd)]
client.upsert(collection_name=srd_collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# Querying

First we make a function to search the vector database for related chunks of text. 

We create an embedding from the query then search for similar embeddings. Returning the chunks of text.

In [9]:
def search_srd(query: str, limit: int = 10):
    # Create embedding from query
    query_embedding = embeddor.embeddings([query])[0]
    # Get the n most similar vectors
    results = client.search(collection_name=srd_collection_name, query_vector=query_embedding, limit=limit, with_payload=True)
    # Request the text for each result
    return [result.payload["text"] for result in results]

Now for the main event the `ask_srd` function. We use the `search_srd` function above to get the top n chunks of text that might be relevant to the query. Then we incorporate those into our prompt to ask GPT-4 to look though them and answer our question. 

In [10]:
def ask_srd(query: str) -> str:
    context = search_srd(query, 20)
    context = "\n".join(context)

    prompt = f"You are a helpful assistant that answers questions about D&D 5e. Given the following excerpts from the rules answer users questions. If you can't find the answer in the excerpts say you don't know.\n{context}"
    system_message = {"role": "system", "content": prompt}
    prompt_message = {"role": "user", "content": query}
    srd_answer = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[system_message, prompt_message],
    )
    
    return srd_answer["choices"][0]["message"]["content"]


Now we can ask whatever questions we want! Your players will never argue with you again when your robot tells them they can't cast two fireballs on the same turn! No sorry that's not how quickened spell works. 

In [11]:
question = "Can I cast two leveled spells on the same turn?"
print(ask_srd(question))

According to the rules regarding bonus action spells, if you cast a spell that has a casting time of 1 bonus action, you can't cast another spell during the same turn, except for a cantrip with a casting time of 1 action.
